In [ ]:
from bs4 import BeautifulSoup
from random import shuffle
import time
import json
import requests as r
import html5lib
import pdfcrowd
import sys
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
userid = os.getenv('USERID')
a_id = os.getenv('A_ID')

In [ ]:
with open('../metadata/Strategy Tools-sub.json', mode = 'r') as f:
    strat = json.loads(f.read())

In [ ]:
def getArticle(article):
    html = r.get(article['link'])
    soup = BeautifulSoup(html.content, features="html5lib")
    body = '<html>' + str(soup.find('body')) + '</html>'

#     delete elements that are not needed

#   check for errors if title contains invalid characters w.r.t. file names
    with open(f"../html/Strategy Tools/{article['title']}.html", mode = 'w',encoding='utf-8') as f:
        f.write(body)

#   return if soup.find('div', attrs = {'id':'article_content'}) exists or not
    return soup.find('div', attrs = {'id':'article_content'}) 

In [ ]:
def generatePDF(fName):
    try:
        inFile = f'../html/Strategy Tools/{fName}.html'
        outFile = f'../pdf/Strategy Tools/{fName}.pdf'
        client = pdfcrowd.HtmlToPdfClient(userid, a_id)
        client.setPageSize('Letter')
        client.setOrientation('landscape')
        client.convertFileToFile(inFile, outFile)
        print('PDF generation complete -',outFile)
    except pdfcrowd.Error as why:
        sys.stderr.write('Pdfcrowd Error: {}\n'.format(why))
        raise

In [ ]:
l = strat[0]['sub-category-content'][7]['link']
print(l)
body = generateHTMLfile(l)

In [ ]:
for sub in strat:
    for article in sub['sub-category-content']:
#         check for 3 articles rule
        stage1 = generateHTMLfile(article)
        if stage1:
#             check for errors if title contains invalid characters w.r.t. file names
            generatePDF(article['title'])